# Графы, обход в глубину и идея OLC-сборки

В этом ноутбуке мы разберём:

1. Что такое граф и как его хранить в Python.
2. Обход графа в глубину (DFS) — базовый алгоритм на графах.
3. Как графы возникают в биоинформатике (особенно в задачах сборки генома).
4. Что такое схема Overlap–Layout–Consensus (OLC) и зачем нужен граф перекрытий.
5. Небольшие практические задания.

Ноутбук рассчитан на то, что вы уже работали с Python, но ещё не писали свои алгоритмы на графах, так что если вы все еще не уверены в своих знаниях python советую посмотреть задачки из simple_python.ipynb

## 1. Что такое граф

**Граф** — это набор объектов (вершин), между которыми есть связи (рёбра). Надеюсь вы помните это с занятия. В биологии часто можно интерпретировать данные в виде графов:
- Вершины: строки ДНК, виды организмов, белки — что угодно.
- Рёбра: "связано с", "переходит в", "перекрывается", "ссылается на", "родственник".

В биоинформатике графы нужны потому что:
- чтения (reads) часто **перекрываются** → можно соединять их в более длинные последовательности;
- иногда один и тот же фрагмент может вести к нескольким продолжениям → удобно хранить в виде графа;
- в сборке генома ранних поколений использовали именно графы перекрытий.
- в новых(относительно конечно) алгоритмах так же используются графы, пускай и более сложные.

В этом ноутбуке мы будем представлять граф как **словарь**: у вершины есть список соседей.
Пример:

```text
A → B, C
B → D
C → (нет)
D → (нет)
```
этому соответствует граф:
```text
   A
  / \
 B   C
 |
 D
 ```

В python это удобно реализовать через словари и списки:

In [ ]:
graph = {
    "A": ["B", "C"],
    "B": ["D"],
    "C": [],
    "D": []
}

graph

## 2. Обход графа в глубину (DFS)

Как вы надеюсь помните с занятия основная идея которая позволила нам придумать как отсортировать граф заключалась в том, чтобы идти по направленному графу до упора, пока не найдем вершину из которой не идет больше ребер. 

**Задача:** обойти все вершины, начиная с какой-то стартовой, при этом в начале пытаться пометить все самые глубокие. 

Идея DFS (Depth-First Search):
1. Берём вершину.
2. "Уходим вглубь" по первому доступному ребру.
3. Когда дальше идти некуда — возвращаемся назад.
4. Чтобы не зациклиться, отмечаем посещённые вершины.

DFS бывает:
- **рекурсивный** (короче, чаще рассматривается),
- **итеративный** (через стек, пожалуй более понятен) 

Мы начнём с рекурсивного, так как поговорили о рекурсии на занятии. 

### Задание 1
1. Допишите функцию `dfs`, чтобы она:
   - добавляла вершину в `visited`,
   - печатала её имя,
   - рекурсивно вызывала `dfs` для всех непосещённых соседей.
2. Проверьте работу на графе из примера.
3. Что будет, если в графе будет цикл? Почему нам нужен `visited`?

In [ ]:
visited = set()

def dfs(graph, node):
    """Обходит граф в глубину, начиная с node.
    graph: словарь {вершина: [соседи]}
    node: стартовая вершина
    """
    # TODO: 1) пометить вершину посещённой
    # TODO: 2) вывести/сохранить её
    # TODO: 3) обойти всех соседей, которые ещё не посещены
    pass

dfs(graph, "A")
print("Посещено:", visited)

In [ ]:
# Граф с циклом, попробуйте посмотреть что происходит если запустить алгоритм без visited
graph_cycle = {
    "A": ["B"],
    "B": ["C"],
    "C": ["A"],  # цикл обратно в A
}

visited = set()
dfs(graph_cycle, "A")
print("Посещено:", visited)

## 3. Зачем всё это в биоинформатике

В сборке генома у нас есть **много относительно коротких прочтений** (reads), которые получились из одной длинной ДНК.
Эти прочтения  **перекрываются** — последние буквы одного совпадают с первыми буквами другого, особенно при линейном секвинировании по сенгеру.

Если мы научимся:
1. наивно находить перекрытия между чтениями(более сложный алгоритм выравнивания мы разберем позже),
2. строить граф "кто может идти после кого",
3. проходить по этому графу в правильном порядке,

то мы сможем собирать более длинные последовательности.

Ранние методы сборки (ещё до повсеместного использования графов де Брейна) часто использовали схему:

**Overlap → Layout → Consensus (OLC)**

- **Overlap** — найти, какие чтения перекрываются.
- **Layout** — расположить чтения в порядке, используя граф.
- **Consensus** — получить финальную строку (учесть ошибки, выбрать наиболее вероятные буквы).

Сегодня мы сделаем маленький кусочек этой схемы: построим **граф перекрытий** и выпрямим его. 

## 4. Граф перекрытий

Пусть у нас есть 3 чтения:

- `ATTAC`
- `TACG`
- `CGT`

Мы говорим, что чтение `ATTAC` может переходить в `TACG`, если **хвост** первого совпадает с **началом** второго.
Например:

- `ATTAC`
     `TACG`
общая часть — `TAC`

Значит, из `ATTAC` есть ребро в `TACG`.

Если мы найдём такие переходы для всех пар чтений — у нас получится **ориентированный граф перекрытий**.
В нём вершины — чтения, рёбра — «это чтение может идти следующим».

Код для поиска перекрытий вам писать не надо, но постарайтесь понять как он работает, если все же хотите написать сами, то посмотрите задачки в simple_python.

In [ ]:
reads = ["ATTAC", "TACG", "CGT", "ACG"]

def overlap(a: str, b: str, min_overlap: int = 2) -> int:
    """
    Возвращает длину максимального перекрытия, когда конец a совпадает с началом b.
    Если перекрытие меньше min_overlap, возвращает 0.
    """
    max_olen = 0
    # перебираем возможные длины перекрытия
    start = 1  # длина перекрытия не может быть 0
    for olen in range(min_overlap, min(len(a), len(b)) + 1):
        if a.endswith(b[:olen]):
            max_olen = olen
    return max_olen

# тест
for a in reads:
    for b in reads:
        if a != b:
            olen = overlap(a, b, min_overlap=2)
            if olen > 0:
                print(f"{a} -> {b} (overlap={olen})")

### Задание 2: построить граф перекрытий

Сейчас вы нашли, какие чтения перекрываются.  
Теперь сделаем из этого граф — в том же формате, что и раньше:

```python
overlap_graph = {
    "ATTAC": ["TACG"],
    "TACG": ["CGT", "ACG"],
    ...
}
```
Ваша задача:
1.	Написать функцию, которая по списку чтений строит словарь read -> [список_чтений_с_перекрытием].
2.	Вывести этот словарь.
3.	(по желанию) визуализировать в виде “A → B”.

In [ ]:
def build_overlap_graph(reads, min_overlap=2):
    graph = {r: [] for r in reads}
    for a in reads:
        for b in reads:
           # TODO: реализовать добавление ребер
           pass
    return graph

overlap_graph = build_overlap_graph(reads, min_overlap=2)
overlap_graph

## 5. Layout: как из графа получить строку

Круто, граф мы получили! Теперь из графа перекрытий нужно понять, 
**в каком порядке должны идти чтения**, чтобы при переходе по рёбрам не нарушать порядок.

Это задача **топологической сортировки**.

**Определение:**  
Для ориентированного ациклического графа (DAG) топологическая сортировка — 
это такой порядок вершин, что для любого ребра `u → v` вершина `u` идёт раньше `v`.

В терминах сборки:
- если чтение `r1` перекрывается с `r2`, то `r1` должно идти **перед** `r2`;
- значит, `r1 → r2` — ребро зависимости.

Если граф не имеет циклов, то мы можем упорядочить чтения однозначно (или почти однозначно).

---
**Пример:**  

Пример:
- ATTAC → TACG (перекрытие `TAC`)
- потом TACG → CGT (перекрытие `CG`)

Строка будет:
`ATTAC` + `G` (от TACG) + `T` (от CGT) = `ATTACGT`


Как же это делать? Тут нам поможет только что просмотренный алгоритм обхода в глубину.
Мы хотим упорядочить чтения так, чтобы для каждого ребра `u → v`
чтение `u` шло раньше `v`.

Это можно сделать при помощи **обхода графа в глубину (DFS)**:
1. Обходим вершину.
2. Для каждого соседа рекурсивно выполняем обход.
3. Когда все соседи уже обработаны — добавляем вершину в конец списка.

В результате вершины, от которых зависят другие, окажутся **раньше**.

Такой способ работает, если граф **ацикличный** (DAG).

---
**Интуиция:**
- Если `A → B`, то B добавится в список *после* A, потому что DFS уходит в B и вернётся обратно только после всех потомков.
- Когда весь обход завершён, нужно просто **перевернуть список**.(или вообще изначально воспринимать это как стек, если вы конечно знаете что это)

In [ ]:
# Задание: Реализуйте рекурсивную топологическую сортировку

# Вход: граф в виде словаря {"вершина": [список_соседей]}
# Выход: список вершин в порядке топологической сортировки

def topo_dfs(node, graph, visited, stack):
    """
    TODO:
    1. Добавьте вершину в visited.
    2. Для каждого соседа, которого ещё нет в visited, вызовите topo_dfs.
    3. После обхода всех соседей добавьте вершину в конец stack.
    """
    pass


def topological_sort_recursive(graph):
    """
    TODO:
    1. Создайте пустое множество visited и пустой список stack.
    2. Для каждой вершины графа, если она ещё не посещена — вызовите topo_dfs.
    3. Разверните stack перед возвратом.
    """
    pass

1. Запустите функцию `topological_sort_recursive(graph)`.
2. Убедитесь, что порядок вершин: `["ATTAC", "TACG", "CGT"]`.
3. Попробуйте добавить ещё одно чтение, например `"CGTA"`, и посмотрите, изменится ли порядок.

In [ ]:
graph = {
    "ATTAC": ["TACG"],
    "TACG": ["CGT"],
    "CGT": []
}

order = topological_sort_recursive(graph)
print("Топологический порядок:", order)

## 6. Сборка по топологическому порядку

Теперь у нас есть список чтений в корректном порядке — можем «склеить» их в одну строку.

Напишите функцию, которая:
1. Берёт первое чтение.
2. Для каждой следующей строки добавляет только неперекрывающуюся часть (как в OLC).

In [ ]:
# Задание: Соберите строку по найденному топологическому порядку

def overlap(a, b, min_overlap=1):
    """Возвращает длину перекрытия конца a и начала b."""
    olen = 0
    for i in range(min_overlap, min(len(a), len(b)) + 1):
        if a.endswith(b[:i]):
            olen = i
    return olen


def assemble_from_order(order, reads):
    """
    TODO:
    1. Начните с первого чтения как seq.
    2. Для каждой следующей вершины:
       - найдите перекрытие с предыдущей (используя overlap),
       - добавьте неперекрывающуюся часть.
    3. Верните собранную строку.
    """
    pass


reads = ["ATTAC", "TACG", "CGT"]
reads_dict = {r: r for r in reads}

assembled = assemble_from_order(order, reads_dict)
print("Собранная последовательность:", assembled)

Теперь вы реализовали «мини-OLC сборщик» с рекурсивным DFS для топологической сортировки, звучит круто и так оно и есть. однако всегда можно сделать больше, в качестве звездочки подумайте над данными вопросами и если хотите напишите небольшой писменный ответ на них:
**На подумать:**
- Что произойдёт, если в графе цикл?
- Как можно модифицировать DFS, чтобы обнаружить такие циклы?
- Почему топологическая сортировка не подходит для графов с циклами?
##### Ваш ответ: 

## На будущее

- **DFS** нужен не только чтобы «обойти все вершины», но и чтобы:
  - проверять связность,
  - находить компоненты,
  - делать топологическую сортировку,
  - искать пути.
- В задачах сборки генома графы бывают двух типов:
  1. **Overlap graph** (как у нас): вершины — чтения, рёбра — перекрытия.
  2. **de Bruijn graph**: вершины — k-меры, рёбра — сдвиг на одну букву. Это более современный и масштабируемый вариант.
- OLC — исторически первый популярный подход для сборки длинных ридов.
- Мы сделали очень упрощённую версию, чтобы показать связь: **строки → перекрытия → граф → обход → сборка.**